# Running SNPGenie on H5N1 data

May 10, 2018 and December 8, 2017

Popoolation is not a great program for our purposes. I have known this for awhile, but it has always suited our needs well enough that I haven't looked elsewhere. However, now that I would like to do some comparisons of calculating pi at different SNP frequency cutoffs, I need something else. Chase Nelson's program, SNPGenie does this and so I am going to try to get that to work for me. There seem to be a few quirks with it that I will need to work around: 

1. SNPGenie only works on 1 gene segment at a time. This is a huge drag. It needs to be run 8 separate times for 1 influenza genome. I also figured out that for each genome, you need to provide a vcf that contains only sites from that gene, a gtf file with only coordinates for that cds, and a reference fasta with only that sequence. 

2. The vcf format options provided are quite close to what I need, but not quite. Varscan output is almost exactly the same as vcf option 4. The only difference is that SNPGenie assumes AD (allele depth) to be the number of reads supporting the reference base, when in varscan, it supports the number of reads for the alternative allele. Additionally, instead of having the reference base reads in 1 column and the alternative allele reads in another, SNPGenie would like them in the same column, separate by a comma. 

3. SNPGenie will not over-write old folders. So if I want everything in 1 folder, I will need to make new names for them as we proceed.

I will need to do several things in order to make SNPGenie work.

1. Fix the AD column issue/reformat my vcfs to accomodate that change. 
2. Automate splitting up vcf files, reference fasta files, and gtf files. 
3. Take care of automatic renaming of folders. 

In [2]:
# import necessary modules
import sys, subprocess, glob, os, shutil, re, importlib, Bio, csv
from subprocess import call
from Bio import SeqIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import rpy2
%load_ext rpy2.ipython 

## 1. Fix AD column issue and split vcf by gene

In [62]:
# set working directory
directory = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/"

In [63]:
# make a function to combine the AD column for vcf files

def format_vcfs_for_snpgenie(vcf_file):
    with open(vcf_file, "r") as csvfile:
        reader = csv.reader(csvfile, delimiter="\t")
        
        # get a list of the genes that have SNPs in the original, combined vcf file
        genes = []
        for row in reader: 
            if "#" not in row[0] and "##" not in row[0]:
                gene = row[0]
                
                if gene.split("_")[-1:] == ['MP']:
                    genes.append(gene.replace("MP","M1"))
                    genes.append(gene.replace("MP","M2"))
                if gene.split("_")[-1:] == ['NS']:
                    genes.append(gene.replace("NS","NS1"))
                    genes.append(gene.replace("NS","NEP"))
                else:
                    genes.append(gene)
        genes = set(genes)
        
        # for each unique gene, make an empty output file with that gene as its name
        for i in genes:
            with open(directory + i + ".vcf", "w") as outfile:
                outfile.write("")
                
    # re-open and re-loop through            
    with open(vcf_file, "r") as infile:
        for line in infile:
            # if it is a comment line, write it out to all of the output files
            if "#" in line or "##" in line:
                for i in genes:
                    with open(directory + i + ".vcf", "a") as outfile:
                        outfile.write(line)
            
            # if it's a SNP line, add in the AD annotation and write out to the appropriate file
            else: 
                gene = line.split("\t")[0]
                fix = line.split("\t")[9]
                ref_reads = fix.split(":")[4]
                alt_reads = fix.split(":")[5]
                new_depth = ref_reads + "," + alt_reads
                first_part = fix.split(":")[0:5]
                first_part = ":".join(first_part)
                second_part = fix.split(":")[6:14]
                second_part = ":".join(second_part)
                fixed = first_part + ":" + new_depth + ":" + second_part
                
                # combine into a new line
                new_line = line.replace(fix, fixed)
                
                if gene.split("_")[-1:] == ['MP']:
                    with open(directory + gene.replace("MP","M1") + ".vcf", "a") as outfile:
                        outfile.write(new_line)
                    with open(directory + gene.replace("MP","M2") + ".vcf", "a") as outfile:
                        outfile.write(new_line)
                        
                elif gene.split("_")[-1:] == ['NS']:
                    with open(directory + gene.replace("NS","NS1") + ".vcf", "a") as outfile:
                        outfile.write(new_line)
                    with open(directory + gene.replace("NS","NEP") + ".vcf", "a") as outfile:
                        outfile.write(new_line)
                  
                else:
                    with open(directory + gene + ".vcf", "a") as outfile:
                        outfile.write(new_line)


In [22]:
import csv
import os

def split_vcf_by_gene(vcf_file, output_directory):
    # Ensure output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Step 1: Identify unique genes
    genes = set()
    with open(vcf_file, "r") as infile:
        for line in infile:
            if not line.startswith("#"):  # Ignore headers for now
                gene = line.split("\t")[0]
                genes.add(gene)

    # Step 2: Create empty files for each gene
    for gene in genes:
        with open(os.path.join(output_directory, f"{gene}.vcf"), "w") as outfile:
            outfile.write("")  # Create empty file

    # Step 3: Read the VCF file again and distribute content
    with open(vcf_file, "r") as infile:
        headers = []
        for line in infile:
            if line.startswith("#"):
                headers.append(line)  # Store headers
            else:
                gene = line.split("\t")[0]
                with open(os.path.join(output_directory, f"{gene}.vcf"), "a") as outfile:
                    outfile.write(line)

        # Step 4: Write headers to all gene-specific files
        for gene in genes:
            with open(os.path.join(output_directory, f"{gene}.vcf"), "r+") as outfile:
                content = outfile.read()
                outfile.seek(0, 0)  # Move cursor to the beginning
                outfile.writelines(headers + [content])  # Write headers first, then content



In [64]:
# go through directory and make a vcf file list of all the files that need to be reformatted

vcfs = []
fastas = []
gtfs = []

for f in glob.glob(directory + "*.vcf"):
    vcfs.append(f)

for f in glob.glob(directory + "*.fasta"):
    fastas.append(f)

gtf_directory = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/"
for f in glob.glob(gtf_directory + "*/reference.gtf"):
    gtfs.append(f)
    

In [65]:

vcfs = glob.glob(directory + "*.vcf")
print("VCF files found:", vcfs)  # <-- This should list all .vcf files

output_subdir = os.path.join(output_directory, os.path.basename(vcf_file).replace(".vcf", ""))
# split_vcf_by_gene(vcf_file, output_subdir)

VCF files found: []


In [29]:
# # run the vcf format function
# output_directory = '/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf'
# output_subdir = os.path.join(output_directory, os.path.basename(vcf_file).replace(".vcf", ""))

# for vcf in vcfs: 
#     vcf_file = vcf
#     split_vcf_by_gene(vcf_file, output_subdir)

In [30]:
import os
import glob

directory = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/"
output_directory = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf/"

# Find all VCF files
vcfs = glob.glob(os.path.join(directory, "*.vcf"))

print("Found VCFs:", vcfs)  # Debugging

if not vcfs:
    print("⚠️ No VCF files found! Check your directory path.")
else:
    for vcf_file in vcfs:
        # Extract the sample name from the filename
        sample_name = os.path.basename(vcf_file).replace(".vcf", "")
        
        # Create a subdirectory for each sample
        sample_output_dir = os.path.join(output_directory, sample_name)
        os.makedirs(sample_output_dir, exist_ok=True)  # Ensure directory exists
        
        print(f"Processing {vcf_file} → Output in {sample_output_dir}")
        
        # Run your function, directing output to the sample-specific folder
        split_vcf_by_gene(vcf_file, sample_output_dir)
        
        print(f"✅ Finished {vcf_file} → Files saved in {sample_output_dir}")


Found VCFs: ['/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w1_replicate-1.vcf', '/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w1_replicate-2.vcf', '/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w2_replicate-1.vcf']
Processing /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w1_replicate-1.vcf → Output in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf/be_w1_replicate-1
✅ Finished /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w1_replicate-1.vcf → Files saved in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf/be_w1_replicate-1
Processing /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_

## Split gtfs and fastas by gene

In [9]:
# split up fasta files

for fasta in fastas:
    with open(fasta, "r") as infile: 
        for seq in SeqIO.parse(infile, "fasta"):
            name = seq.description
            sequence = str(seq.seq)
        
            if name.endswith("MP"):
                with open(directory + name.replace("MP","M1") + ".fasta", "w") as outfile:
                    outfile.write(">" + name.replace("MP","M1") + "\n")
                    outfile.write(sequence) 
                with open(directory + name.replace("MP","M2") + ".fasta", "w") as outfile:
                    outfile.write(">" + name.replace("MP","M2") + "\n")
                    outfile.write(sequence)              
                    
            elif name.endswith("NS"):
                with open(directory + name.replace("NS","NS1") + ".fasta", "w") as outfile:
                    outfile.write(">" + name.replace("NS","NS1") + "\n")
                    outfile.write(sequence) 
                with open(directory + name.replace("NS","NEP") + ".fasta", "w") as outfile:
                    outfile.write(">" + name.replace("NS","NEP") + "\n")
                    outfile.write(sequence) 
                    
            else:
                with open(directory + name + ".fasta", "w") as outfile:
                    outfile.write(">" + name + "\n")
                    outfile.write(sequence) 

In [38]:
# split up gtf files 

for gtf in gtfs: 
    with open(gtf, "r") as infile:
        for line in infile:
            gene = line.split("\t")[0]
        
            if gene.endswith("MP") and "\"M1\"" in line:
                with open(directory + gene.replace("MP","M1") + ".gtf", "a") as outfile:
                        outfile.write(line)
            
            elif gene.endswith("MP") and "\"M2\"" in line:
                with open(directory + gene.replace("MP","M2") + ".gtf", "a") as outfile:
                    outfile.write(line)
                    
            elif gene.endswith("NS") and "\"NS1\"" in line:
                with open(directory + gene.replace("NS","NS1") + ".gtf", "a") as outfile:
                    outfile.write(line)
            
            elif gene.endswith("NS") and "\"NEP\"" in line:
                with open(directory + gene.replace("NS","NEP") + ".gtf", "a") as outfile:
                    outfile.write(line)
                    
            else:
                with open(directory + gene + ".gtf", "a") as outfile:
                    outfile.write(line)

In [45]:
##renaming reference gtfs and fastas

import os

base_directory = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference"  # Change this to your main directory

for root, _, files in os.walk(base_directory):
    subdir_name = os.path.basename(root)  # Get the subdirectory name

    for filename in files:
        if filename.startswith("."):  # Ignore hidden/system files
            continue

        file_extension = os.path.splitext(filename)[-1]  # Extract file extension
        
        if "reference" in filename:
            new_name = filename.replace("reference", subdir_name)  # Replace 'reference' with subdirectory name
            old_path = os.path.join(root, filename)
            new_path = os.path.join(root, new_name)

            os.rename(old_path, new_path)
            print(f"Renamed in {root}: {filename} → {new_name}")


Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/ns: reference.gb → ns.gb
Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/ns: reference.gtf → ns.gtf
Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/ns: reference.fasta → ns.fasta
Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/na: reference.gb → na.gb
Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/na: reference.gtf → na.gtf
Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/na: reference.fasta → na.fasta
Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/reference/pb2: reference.gb → pb2.gb
Renamed in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SN

In [52]:
##renaming parsed vcfs to have the sample name so they can be all in one directory

# Set the parent directory containing sample subdirectories
parent_directory = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf"  # Change this!

# Loop through each subdirectory
for sample in os.listdir(parent_directory):
    sample_path = os.path.join(parent_directory, sample)
    
    # Ensure it's a directory
    if os.path.isdir(sample_path):
        print(f"📂 Processing: {sample}")

        # Loop through each file in the sample's directory
        for filename in os.listdir(sample_path):
            old_file_path = os.path.join(sample_path, filename)
            
            # Construct the new filename
            new_filename = f"{sample}_{filename}"
            new_file_path = os.path.join(sample_path, new_filename)

            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"✅ Renamed: {filename} → {new_filename}")

print("🎉 Done! All files renamed.")


📂 Processing: be_w1_replicate-2
✅ Renamed: np.vcf → be_w1_replicate-2_np.vcf
✅ Renamed: ns.vcf → be_w1_replicate-2_ns.vcf
✅ Renamed: na.vcf → be_w1_replicate-2_na.vcf
✅ Renamed: pa.vcf → be_w1_replicate-2_pa.vcf
✅ Renamed: pb2.vcf → be_w1_replicate-2_pb2.vcf
✅ Renamed: ha.vcf → be_w1_replicate-2_ha.vcf
✅ Renamed: pb1.vcf → be_w1_replicate-2_pb1.vcf
📂 Processing: gtf
✅ Renamed: pa.gtf → gtf_pa.gtf
✅ Renamed: pb1.gtf → gtf_pb1.gtf
✅ Renamed: ha.gtf → gtf_ha.gtf
✅ Renamed: pb2.gtf → gtf_pb2.gtf
✅ Renamed: ns.gtf → gtf_ns.gtf
✅ Renamed: np.gtf → gtf_np.gtf
✅ Renamed: na.gtf → gtf_na.gtf
📂 Processing: fasta
✅ Renamed: pa.fasta → fasta_pa.fasta
✅ Renamed: ha.fasta → fasta_ha.fasta
✅ Renamed: na.fasta → fasta_na.fasta
✅ Renamed: pb2.fasta → fasta_pb2.fasta
✅ Renamed: np.fasta → fasta_np.fasta
✅ Renamed: ns.fasta → fasta_ns.fasta
✅ Renamed: pb1.fasta → fasta_pb1.fasta
📂 Processing: be_w1_replicate-1
✅ Renamed: np.vcf → be_w1_replicate-1_np.vcf
✅ Renamed: ns.vcf → be_w1_replicate-1_ns.vcf
✅ Ren

### Consolidate files together into folders by sample name

In [48]:
# make a list of file names, which are just the gene names that the gtf file, fasta file and vcf files share
file_list = []
for f in glob.glob(output_directory + "*.vcf"):
    f = f.replace(".vcf","")
    file_list.append(f)

In [50]:
output_directory

'/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf/'

In [53]:
import glob
import os

file_list = []

# Search for all VCF files within subdirectories of output_directory
for f in glob.glob(os.path.join(output_directory, "**", "*.vcf"), recursive=True):
    # Extract the filename without extension
    sample_name = os.path.basename(f).replace(".vcf", "")
    file_list.append(sample_name)

print(file_list)


['be_w1_replicate-2_pa', 'be_w1_replicate-2_pb1', 'be_w1_replicate-2_ha', 'be_w1_replicate-2_pb2', 'be_w1_replicate-2_np', 'be_w1_replicate-2_ns', 'be_w1_replicate-2_na', 'be_w1_replicate-1_ha', 'be_w1_replicate-1_pa', 'be_w1_replicate-1_na', 'be_w1_replicate-1_np', 'be_w1_replicate-1_ns', 'be_w1_replicate-1_pb2', 'be_w1_replicate-1_pb1', 'be_w2_replicate-1_pa', 'be_w2_replicate-1_ha', 'be_w2_replicate-1_ns', 'be_w2_replicate-1_np', 'be_w2_replicate-1_na', 'be_w2_replicate-1_pb1', 'be_w2_replicate-1_pb2']


In [61]:
import os
from subprocess import call

# Define the specific output directory where results will go
output_directory = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/"
os.makedirs(output_directory, exist_ok=True)

# Define the base path for the reference FASTA and GTF files
reference_base_path = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf/reference/"

# Define the mapping between sample suffixes and reference file names
reference_mapping = {
    'pa': 'pa',  # Reference files for 'pa' samples
    'pb1': 'pb1',  # Reference files for 'pb1' samples
    'pb2': 'pb2',  # Reference files for 'pb2' samples
    'ha': 'ha',  # Reference files for 'ha' samples
    'np': 'np',  # Reference files for 'np' samples
    'ns': 'ns',  # Reference files for 'ns' samples
    'na': 'na',  # Reference files for 'na' samples
}

# Run SNPGenie for each sample
for sample in file_list:
    # Extract the sample suffix (e.g., 'pa', 'pb1', 'ha' etc. from sample name)
    sample_suffix = sample.split('_')[-1]  # Assuming the format ends with 'pa', 'pb1', etc.
    
    # Check if the sample_suffix has a valid mapping
    if sample_suffix in reference_mapping:
        reference_name = reference_mapping[sample_suffix]
        
        # Construct the paths to the reference FASTA and GTF files
        reference_fasta = os.path.join(reference_base_path, f"{reference_name}.fasta")
        reference_gtf = os.path.join(reference_base_path, f"{reference_name}.gtf")

        # Check if the reference files exist (optional, for error handling)
        if not os.path.exists(reference_fasta) or not os.path.exists(reference_gtf):
            print(f"Warning: Missing reference files for {sample_suffix}. Skipping {sample}.")
            continue
        
        # Construct the VCF file path (assuming the VCF file is named after the sample)
        vcf_file = os.path.join('/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/parsed_vcf/', f"{sample}.vcf")  # Adjust path if needed
        
        # Check if the VCF file exists
        if not os.path.exists(vcf_file):
            print(f"VCF file missing for sample {sample}. Skipping.")
            continue

        # Define the SNPGenie output directory for the sample
        snpgenie_output_dir = os.path.join(output_directory, f"{sample}_results/")
        os.makedirs(snpgenie_output_dir, exist_ok=True)

        # Run SNPGenie using the corresponding reference and the sample's VCF file
        snpgenie_command = f"perl /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/snpgenie.pl --vcfformat=4 --gtffile={reference_gtf} --fastafile={reference_fasta} --snpreport={snpgenie_output_dir}{sample}_snpgenie.vcf"
        
        # Execute the SNPGenie command
        print(f"Running SNPGenie for {sample}...")
        call(snpgenie_command, shell=True)

        # Move SNPGenie results to the specific output folder (if applicable)
        # Assuming SNPGenie generates a directory named SNPGenie_Results by default
        call(f"mv SNPGenie_Results {snpgenie_output_dir}", shell=True)
        
        print(f"Results for {sample} saved in {snpgenie_output_dir}")
    else:
        print(f"No reference mapping found for sample suffix {sample_suffix}. Skipping.")


Running SNPGenie for be_w1_replicate-2_pa...


################################################################################
##                                                                            ##
##                             SNPGenie Initiated!                            ##
##                                                                            ##
################################################################################

  ###############################  LICENSE:  #################################
  ##            SNPGenie Copyright (C) 2015-19 Chase W. Nelson              ##
  ##            This program comes with ABSOLUTELY NO WARRANTY;             ##
  ##     This is free software, and you are welcome to redistribute it      ##
  ##               under certain conditions; see LICENSE.txt.               ##
  ############################################################################

WORKING_DIRECTORY=/Users/asjaeger/Desktop/project_source/hpai_PA
RESULTS



## WARNING: The CDS coordinates for gene PA-X in the gtf file do not yield a set of complete codons,
## or are absent from the file. The number of nucleotides must be a multiple of 3.
## SNPGenie terminated.



## WARNING:
## The SNP Report /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w1_replicate-2_pb1_results/be_w1_replicate-2_pb1_snpgenie.vcf has no header.
## TERMINATED.





################################################################################
##                                                                            ##
##                             SNPGenie Initiated!                            ##
##                                                                            ##
################################################################################

  ###############################  LICENSE:  #################################
  ##            SNPGenie Copyright (C) 2015-19 Chase W. Nelson              ##
  ##            This program comes with ABSOLUTELY NO WARRANTY;             ##
  ##     This is free software, and you are welcome to redistribute it      ##
  ##               under certain conditions; see LICENSE.txt.               ##
  ############################################################################

WORKING_DIRECTORY=/Users/asjaeger/Desktop/project_source/hpai_PA
RESULTS_DIRECTORY=/Users/asjaeger/Desktop/project_so



## WARNING:
## The SNP Report /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w1_replicate-2_ha_results/be_w1_replicate-2_ha_snpgenie.vcf has no header.
## TERMINATED.



## WARNING:
## The SNP Report /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w1_replicate-2_pb2_results/be_w1_replicate-2_pb2_snpgenie.vcf has no header.
## TERMINATED.





################################################################################
##                                                                            ##
##                             SNPGenie Initiated!                            ##
##                                                                            ##
################################################################################

  ###############################  LICENSE:  #################################
  ##            SNPGenie Copyright (C) 2015-19 Chase W. Nelson              ##
  ##            This program comes with ABSOLUTELY NO WARRANTY;             ##
  ##     This is free software, and you are welcome to redistribute it      ##
  ##               under certain conditions; see LICENSE.txt.               ##
  ############################################################################

WORKING_DIRECTORY=/Users/asjaeger/Desktop/project_source/hpai_PA
RESULTS_DIRECTORY=/Users/asjaeger/Desktop/project_so

...



## WARNING:
## The SNP Report /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w2_replicate-1_ha_results/be_w2_replicate-1_ha_snpgenie.vcf has no header.
## TERMINATED.



## WARNING: The CDS coordinates for gene NEP in the gtf file do not yield a set of complete codons,
## or are absent from the file. The number of nucleotides must be a multiple of 3.
## SNPGenie terminated.



Results for be_w2_replicate-1_ha saved in /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w2_replicate-1_ha_results/
Running SNPGenie for be_w2_replicate-1_ns...


################################################################################
##                                                                            ##
##                             SNPGenie Initiated!                            ##
##                                                                            ##
################################################################################

  ###############################  LICENSE:  #################################
  ##            SNPGenie Copyright (C) 2015-19 Chase W. Nelson              ##
  ##            This program comes with ABSOLUTELY NO WARRANTY;             ##
  ##     This is free software, and you are welcome to redistribute it      ##
  ##               under certain conditions; see LICENSE.txt.               #



## WARNING: The CDS coordinates for gene NP in the gtf file do not yield a set of complete codons,
## or are absent from the file. The number of nucleotides must be a multiple of 3.
## SNPGenie terminated.



## WARNING:
## The SNP Report /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w2_replicate-1_na_results/be_w2_replicate-1_na_snpgenie.vcf has no header.
## TERMINATED.





################################################################################
##                                                                            ##
##                             SNPGenie Initiated!                            ##
##                                                                            ##
################################################################################

  ###############################  LICENSE:  #################################
  ##            SNPGenie Copyright (C) 2015-19 Chase W. Nelson              ##
  ##            This program comes with ABSOLUTELY NO WARRANTY;             ##
  ##     This is free software, and you are welcome to redistribute it      ##
  ##               under certain conditions; see LICENSE.txt.               ##
  ############################################################################

WORKING_DIRECTORY=/Users/asjaeger/Desktop/project_source/hpai_PA
RESULTS_DIRECTORY=/Users/asjaeger/Desktop/project_so



## WARNING:
## The SNP Report /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w2_replicate-1_pb1_results/be_w2_replicate-1_pb1_snpgenie.vcf has no header.
## TERMINATED.



## WARNING:
## The SNP Report /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/be_w2_replicate-1_pb2_results/be_w2_replicate-1_pb2_snpgenie.vcf has no header.
## TERMINATED.



In [54]:
# consolidate fastas, vcfs, and gtfs all into 1 folder together 
for f in file_list:
    call("mkdir {f}".format(f=f), shell=True)
    for n in glob.glob(directory + "*"):
        if f in n:
            call("mv {n} {f}".format(n=n,f=f), shell=True)

In [56]:
# Define the specific path where you want the new directories to be created
new_dir_path = "/Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/"

# Consolidate fastas, vcfs, and gtfs all into the specific folder
for f in file_list:
    # Create the directory in the specific path
    call("mkdir -p {}/{}".format(new_dir_path, f), shell=True)

    # Move files to the new directory
    for n in glob.glob(directory + "*"):
        if f in n:
            call("mv {n} {}/{}/".format(new_dir_path, f, n), shell=True)


#### Note: 

At this point, there will be gtf and fasta files for genes that are not in folders. That is ok. That means that there were no SNPs called for that gene in that sample. 

In [57]:
file_list

['be_w1_replicate-2_pa',
 'be_w1_replicate-2_pb1',
 'be_w1_replicate-2_ha',
 'be_w1_replicate-2_pb2',
 'be_w1_replicate-2_np',
 'be_w1_replicate-2_ns',
 'be_w1_replicate-2_na',
 'be_w1_replicate-1_ha',
 'be_w1_replicate-1_pa',
 'be_w1_replicate-1_na',
 'be_w1_replicate-1_np',
 'be_w1_replicate-1_ns',
 'be_w1_replicate-1_pb2',
 'be_w1_replicate-1_pb1',
 'be_w2_replicate-1_pa',
 'be_w2_replicate-1_ha',
 'be_w2_replicate-1_ns',
 'be_w2_replicate-1_np',
 'be_w2_replicate-1_na',
 'be_w2_replicate-1_pb1',
 'be_w2_replicate-1_pb2']

In [ ]:
import os
from subprocess import call

# Define the specific output directory where results will go
output_directory = "/desired/output/path/"
os.makedirs(output_directory, exist_ok=True)

# Define paths to the reference FASTA and GTF files (they are gene-specific, not sample-specific)
reference_fasta = "/path/to/reference/fastas/reference.fasta"
reference_gtf = "/path/to/reference/gtfs/reference.gtf"

# Run SNPGenie for each sample
for f in file_list:
    # Extract the sample name (assuming 'f' is the path to the sample directory)
    sample_name = os.path.basename(f)

    # Path to the sample's VCF file
    vcf_file = os.path.join(f, "vcfs", f"{sample_name}.vcf")  # Adjust as needed to match exact file names

    # Ensure the VCF file exists
    if os.path.exists(vcf_file):
        # Run SNPGenie using the reference files for each sample
        snpgenie_command = f"/Users/asjaeger/Desktop/project_source/hpai_PA/snpgenie.pl --vcfformat=4 --gtffile={reference_gtf} --fastafile={reference_fasta} --snpreport={output_directory}{sample_name}_results.vcf"
        
        # Execute the SNPGenie command
        call(snpgenie_command, shell=True)

        # Move the SNPGenie results to the output directory (if they aren't already there)
        call(f"mv SNPGenie_Results {output_directory}{sample_name}_results/", shell=True)
    else:
        print(f"VCF file missing for sample {sample_name}. Skipping SNPGenie.")


In [35]:
# run SNP Genie on all of these
for f in file_list:
    f1 = f.replace(output_directory, "")
    call("/Users/asjaeger/Desktop/project_source/hpai_PA/snpgenie.pl --vcfformat=4 --gtffile={f}/{f1}.gtf --fastafile={f}/{f1}.fasta --snpreport={f}/{f1}.vcf".format(f=f,f1=f1),shell=True)
    call("mv SNPGenie_Results {f}/".format(f=f,f1=f1), shell=True)

/bin/sh: /Users/asjaeger/Desktop/project_source/hpai_PA/snpgenie.pl: No such file or directory
mv: rename SNPGenie_Results to /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w1_replicate-1/SNPGenie_Results: No such file or directory
/bin/sh: /Users/asjaeger/Desktop/project_source/hpai_PA/snpgenie.pl: No such file or directory
mv: rename SNPGenie_Results to /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w1_replicate-2/SNPGenie_Results: No such file or directory
/bin/sh: /Users/asjaeger/Desktop/project_source/hpai_PA/snpgenie.pl: No such file or directory
mv: rename SNPGenie_Results to /Users/asjaeger/Desktop/project_source/hpai_PA/seq/illumina-pipeline/for_SNPgenie/sample_vcfs_test/be_w2_replicate-1/SNPGenie_Results: No such file or directory


In [352]:
# compile all of the output files from the various genes
call("grep -i '' */SNPGenie_Results/product_results.txt >> combined.txt".format(directory=directory),shell=True)

0

In [583]:
# take out all of the unwanted lines from the combined file
def cleanup_snpgenie_output(directory, infile):
    with open(directory + infile, "r") as infile: 
        linenumber = 0
        for line in infile:
            linenumber += 1
            if linenumber == 1:
                new_line = line.split(":")[1]
                with open(directory + "combined_pi_formatted.txt", "a") as outfile:
                    outfile.write(new_line)
        
            elif "N_diffs" not in line:
                new_line = line.replace("/SNPGenie_Results/product_results.txt:temp_vcf4_Sample1.vcf","")
            
                genes = ["_PB2","_PB1","_PA","_H5","_NP","_N1","_M1","_M2","_NS1","_NEP"]
                for g in genes: 
                    if g + "\t" in new_line:
                        new_line2 = new_line.replace(g, "")
                with open(directory + "combined_pi_formatted.txt", "a") as outfile:
                    outfile.write(new_line2)

In [584]:
cleanup_snpgenie_output(directory, "combined.txt")

## Results:

This actually all worked really well! Except for the grep, which for some reason works totally fine in shell but isn't working for me when run from within this notebook. Otherwise though, things look good! I will now go through and see which genes didn't work/have empty values. 

## Data wrangling and plotting:

In [585]:
# read in the SNPGenie pi results as a dataframe
df = pd.read_table(directory + "combined_pi_formatted.txt", sep="\t")
df['product'] = df['product'].fillna('neuram')                                     # replace NA with neuram
df['species'] = np.where((df['file'].str.contains("duck")), "duck","human")        # add in a species column
df.head()

,file,product,N_diffs,S_diffs,N_diffs_vs_ref,S_diffs_vs_ref,N_sites,S_sites,piN,piS,mean_dN_vs_ref,mean_dS_vs_ref,mean_gdiv_polymorphic,mean_N_gdiv,mean_S_gdiv,species
0,AH7E5L724F516_A_duck_Cambodia_Y0224304_2014,HA,0.118954,0.403580,0.063241,0.278997,1328.854414,372.145586,0.000090,0.001084,0.000048,0.000750,0.260399283550513,0.118483338280554,0.402315228820471,duck
1,AH7E5L724F516_A_duck_Cambodia_Y0224304_2014,NP,0.235276,0.445634,0.122255,0.235606,1145.707418,348.292582,0.000205,0.001279,0.000107,0.000676,0.0847834938232775,0.0781347069643767,0.0887727659386179,duck
2,AH7E5L724F516_A_duck_Cambodia_Y0224304_2014,PB2,0.000000,0.150964,0.000000,0.081633,1751.833333,525.166667,0.000000,0.000287,0.000000,0.000155,0.14993752603082,*,0.14993752603082,duck
3,AJ4MBL718F513_A_CAMBODIA_V0417301_2011,HA,0.549191,0.280857,0.312999,0.151515,1332.785015,371.214985,0.000412,0.000757,0.000235,0.000408,0.137771377959253,0.136868269877355,0.139577594123049,human
4,AJ4MBL718F513_A_CAMBODIA_V0417301_2011,M1,0.000000,0.000000,0.000000,0.000000,576.666667,179.333333,0.000000,0.000000,0.000000,0.000000,*,*,*,human


In [586]:
# cast the dataframe and calculate some mean values for piN 
pN_h = df[df['species'] == 'human']
pN_h = pN_h[['file','product','piN']]
pN_h = pN_h.pivot(index='file', columns='product', values='piN')
pN_h.index.name = None
pN_h.columns.name = None
pN_h = pN_h.fillna(0)
pN_h.loc['mean'] = pN_h.mean()
pN_h.loc['sd'] = pN_h.std()
pN_means_h = pN_h.loc[['mean','sd']]
pN_means_h = pN_means_h.transpose()
pN_means_h["measure"] = "piN"
pN_means_h["species"] = "human"

pN_d = df[df['species'] == 'duck']
pN_d = pN_d[['file','product','piN']]
pN_d = pN_d.pivot(index='file', columns='product', values='piN')
pN_d.index.name = None
pN_d.columns.name = None
pN_d = pN_d.fillna(0)
pN_d.loc['mean'] = pN_d.mean()
pN_d.loc['sd'] = pN_d.std()
pN_means_d = pN_d.loc[['mean','sd']]
pN_means_d = pN_means_d.transpose()
pN_means_d["measure"] = "piN"
pN_means_d["species"] = "duck"
pN_means_d.head()

,mean,sd,measure,species
HA,0.000045,0.000051,piN,duck
M1,0.000115,0.000131,piN,duck
M2,0.000065,0.000144,piN,duck
NEP,0.000025,0.000057,piN,duck
NP,0.000099,0.000108,piN,duck


In [587]:
# cast the dataframe and calculate some mean values for piN 
pS_h = df[df['species'] == 'human']
pS_h = pS_h[['file','product','piS']]
pS_h = pS_h.pivot(index='file', columns='product', values='piS')
pS_h.index.name = None
pS_h.columns.name = None
pS_h = pS_h.fillna(0)
pS_h.loc['mean'] = pS_h.mean()
pS_h.loc['sd'] = pS_h.std()
pS_means_h = pS_h.loc[['mean','sd']]
pS_means_h = pS_means_h.transpose()
pS_means_h["measure"] = "piS"
pS_means_h["species"] = "human"

pS_d = df[df['species'] == 'duck']
pS_d = pS_d[['file','product','piS']]
pS_d = pS_d.pivot(index='file', columns='product', values='piS')
pS_d.index.name = None
pS_d.columns.name = None
pS_d = pS_d.fillna(0)
pS_d.loc['mean'] = pS_d.mean()
pS_d.loc['sd'] = pS_d.std()
pS_means_d = pS_d.loc[['mean','sd']]
pS_means_d = pS_means_d.transpose()
pS_means_d["measure"] = "piS"
pS_means_d["species"] = "duck"
pS_means_d.head()

,mean,sd,measure,species
HA,0.000207,0.000397,piS,duck
M1,0.000000,0.000000,piS,duck
M2,0.000000,0.000000,piS,duck
NEP,0.000262,0.000380,piS,duck
NP,0.000237,0.000469,piS,duck


In [588]:
# combine the mean and stdev values for piN and piS into 1 dataframe 
df1 = pd.concat([pN_means_h, pN_means_d, pS_means_h, pS_means_d])
df1.reset_index(inplace=True)
df1.head()

,index,mean,sd,measure,species
0,HA,0.000253,0.000260,piN,human
1,M1,0.000033,0.000051,piN,human
2,M2,0.000123,0.000175,piN,human
3,NEP,0.000051,0.000113,piN,human
4,NP,0.000036,0.000055,piN,human


In [377]:
%%R -w 800 -h 500 -u px -i df1  # this sets the size of the plot...otherwise, it will go off the page
require(ggplot2)
library(ggplot2)

df1$index = gsub("neuram", "NA", df1$index)
df1$gene = factor(df1$index, levels=c("PB2","PB1","PA","HA","NP","NA","M1","M2","NS1","NEP"))

p <- ggplot(data=df1, aes(x=gene, y=mean, fill=species)) + 
    geom_col(position="dodge", stat="identity")+
    geom_errorbar(data=df1, aes(x=gene, ymin=mean-sd, ymax=mean+sd, color=species),position=position_dodge(0.9), stat="identity")+
    facet_wrap(~measure, scales="free")+
    scale_fill_manual(values=c("#99bfaa","#5c3d46"))+
    scale_color_manual(values=c("#99bfaa","#5c3d46"))+
    labs(x="gene",y="diversity")+
    #ggtitle("number of shared sites in random simulations") + 
    scale_y_continuous(limits=c(-0.0006,0.002))+
    #scale_x_continuous(limits=c(0,500))+
    theme(plot.title = element_text(size=20, hjust=0.5))+
    theme(panel.grid.major.y=element_line(colour=NA))+
    theme(panel.grid.minor=element_line(colour=NA,size=NA))+    
    theme(strip.background = element_rect(colour=NA, fill=NA))+
    theme(axis.line.x=element_line(colour="black"))+
    theme(axis.line.y=element_line(colour="black"))+theme(strip.text.x=element_text(size=11))+
    theme(axis.title.y=element_text(size=16, vjust=8))+
    theme(axis.title.x=element_text(size=16, vjust=-8))+
    theme(axis.text=element_text(size=16, colour="black"))+
    theme(axis.text.x=element_text(size=16, angle=90))+
    theme(legend.text=element_text(size=16))+theme(legend.title=element_text(size=16, face="plain"))+
    theme(panel.margin=unit(1, "lines"))+theme(plot.margin=unit(c(1,1,1,1),"cm"))+
    theme(legend.key.size=unit(0.7, "cm"))+
    theme(panel.background=element_rect(fill=NA))+
    theme(legend.key=element_rect(fill=NA))

ggsave("piN_and_piS_means-2018-05-10.pdf", p, width = 10, height = 5, device=pdf, path="/Users/lmoncla/Documents/H5N1_Cambodian_outbreak_study/paper-and-figure-drafts/figures-2018-05-08")

/Users/lmoncla/anaconda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning:
  warnings.warn(x, RRuntimeWarning)
/Users/lmoncla/anaconda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning:  Ignoring unknown parameters: stat

  warnings.warn(x, RRuntimeWarning)


## Plot number of SNPs/site

I am starting to think that maybe using pi for this dataset isn't so great. I think that I am going to try out and see how it looks to just plot the overall number of nonsynonymous and synonymous SNPs per nonsynonymous or synonymou site per gene per species. I can read in the number of SNPs from the combined SNPs output file. For the number of synonymous and nonsynonymous sites, I need to count the samples in df (those that had a snp) but also those that did not. I think that the easiest way to do that will be to just run SNPGenie on a set of dummy data so that I can get the number of synonymous and nonsynonymous sites per gene per sample.  

### Step 1: Calculate the average number of synonymous and nonsynonymous sites per gene 

This will involve using the ones I already have plus generating fake vcf files so that I can run SNPGenie for the samples/genes that didn't have any SNPs. 


#### Generate dummy vcfs to get the number of nonsynonymous and synonymous sites for genes that don't have snps

In [589]:
missing_vcfs_directory = "/Users/lmoncla/Documents/H5N1_Cambodian_outbreak_study/SNPGenie_diversity_estimates/1_percent_SNPs-2018-05-10/samples_and_genes_with_no_snps/"

In [590]:
samples = []
for f in glob.glob(missing_vcfs_directory + "*.gtf"):
    files.append(f)
    sample_name = f.replace(missing_vcfs_directory, "")
    sample_name = sample_name.replace(".gtf","")
    samples.append(sample_name)

In [591]:
# make fake vcf files for each sample 
for s in samples: 
    vcfname = missing_vcfs_directory + s + ".vcf"
    with open(vcfname, "w") as vcf: 
        vcf.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSample1\n")
        vcf.write(s+"\t28\t.\tC\tT\t.\tPASS\tADP=147;WT=1000;HET=1;HOM=0;NC=0\tGT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR\t0/1:37:147:147:135:135,12:8.16%:1.9326E-4:38:37:61:74:5:7\n")
        vcf.write(s+"\t100\t.\tC\tT\t.\tPASS\tADP=147;WT=1000;HET=1;HOM=0;NC=0\tGT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR\t0/1:37:147:147:135:135,12:8.16%:1.9326E-4:38:37:61:74:5:7")

In [593]:
# consolidate fastas, vcfs, and gtfs all into 1 folder together 
for s in samples:
    call("mkdir {missing_vcfs_directory}{s}".format(missing_vcfs_directory=missing_vcfs_directory,s=s), shell=True)
    for n in glob.glob(missing_vcfs_directory + "*"):
        if s in n:
            call("mv {n} {missing_vcfs_directory}{s}".format(s=s, n=n,missing_vcfs_directory=missing_vcfs_directory), shell=True)

In [594]:
# run SNPgenie
for s in samples:
    call("snpgenie.pl --vcfformat=4 --gtffile={missing_vcfs_directory}{s}/{s}.gtf --fastafile={missing_vcfs_directory}{s}/{s}.fasta --snpreport={missing_vcfs_directory}{s}/{s}.vcf".format(missing_vcfs_directory=missing_vcfs_directory,s=s),shell=True)
    call("mv SNPGenie_Results {missing_vcfs_directory}{s}/".format(missing_vcfs_directory=missing_vcfs_directory,s=s), shell=True)

In [ ]:
# compile all of the output files from the various genes
call("grep -i '' */SNPGenie_Results/product_results.txt >> combined.txt".format(directory=directory),shell=True)

In [595]:
# clean up outfile
cleanup_snpgenie_output(missing_vcfs_directory, "combined.txt")

In [605]:
# read in the SNPGenie pi results as a dataframe
df_dummy_values = pd.read_table(missing_vcfs_directory + "combined_pi_formatted.txt", sep="\t")
df_dummy_values['product'] = df['product'].fillna('neuram')                                     # replace NA with neuram
df_dummy_values['species'] = np.where((df_dummy_values['file'].str.contains("duck")), "duck","human")        # add in a species column
df_dummy_values = df_dummy_values[['file','product','N_sites','S_sites','species']]
df_dummy_values = df_dummy_values.rename(index=str, columns={"file":'sample','product':'gene'})
df_dummy_values.head()
print(len(df_dummy_values))

42


In [606]:
# read in the values from the genes that did have SNPs
sites = df[['file','product','N_sites','S_sites','species']]
sites = sites.rename(index=str, columns={'file':'sample','product':'gene'})
sites.head()
print(len(sites))

78


In [607]:
# combine dummy and real data into 1 dataframe
sites2 = pd.concat([sites,df_dummy_values])
sites2 = sites2[['gene','N_sites','S_sites','species']]
sites2.head()

,gene,N_sites,S_sites,species
0,HA,1328.854414,372.145586,duck
1,NP,1145.707418,348.292582,duck
2,PB2,1751.833333,525.166667,duck
3,HA,1332.785015,371.214985,human
4,M1,576.666667,179.333333,human


In [608]:
# calculate the mean number of nonsynonymous and synonymous sites per gene per species and output that into a dataframe
genes = ['PB2','PB1','PA','HA','NP','neuram','M1','M2','NS1','NEP']
species = ['duck','human']
count = 0

for g in genes: 
    for s in species:
        count += 1
        subsetdf = sites2.loc[(sites2['gene'] == g) & (sites2['species'] == s)]
        subsetdf.loc['mean'] =  subsetdf.mean()
        subsetdf['gene'] = g
        subsetdf['species'] = s
        subsetdf = subsetdf.loc[['mean']]
        
        if count == 1:
            mean_sites_df = subsetdf
        else:
            mean_sites_df = pd.concat([subsetdf, means_df])

mean_sites_df['gene'] = mean_sites_df['gene'].str.replace("neuram","NA")
mean_sites_df.head()

/Users/lmoncla/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/lmoncla/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/lmoncla/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,gene,N_sites,S_sites,species
mean,NEP,286.101039,76.898961,human
mean,NEP,286.101039,76.898961,human
mean,NEP,285.992593,77.007407,duck
mean,NS1,520.724740,154.275260,human
mean,NS1,521.000000,154.000000,duck


In [609]:
# read in snps csv file and count the number of nonsynonymous and synonymous per gene per species

with open(snps_path, "r") as infile: 
    snps_dict = {}
    for line in infile: 
        if "reference_position" not in line:
            sample = line.split("\t")[0]
            sample = "_".join(sample.split("_")[:-1])
        
            if "duck" in sample:
                species = "duck"
            else:
                species = "human"
        
            gene = line.split("\t")[1]
            syn_nonsyn = line.split("\t")[6]

            if species not in snps_dict:
                snps_dict[species] = {}
            if gene not in snps_dict[species]:
                snps_dict[species][gene] = {"synonymous":0, "nonsynonymous":0}
            if syn_nonsyn == "nonsynonymous":
                snps_dict[species][gene]["nonsynonymous"] += 1
            elif syn_nonsyn == "synonymous":
                snps_dict[species][gene]["synonymous"] += 1
                
print(snps_dict)

{'duck': {'M1': {'synonymous': 0, 'nonsynonymous': 4}, 'M2': {'synonymous': 0, 'nonsynonymous': 1}, 'NA': {'synonymous': 2, 'nonsynonymous': 2}, 'NP': {'synonymous': 6, 'nonsynonymous': 6}, 'NS1': {'synonymous': 1, 'nonsynonymous': 3}, 'NEP': {'synonymous': 2, 'nonsynonymous': 1}, 'PA': {'synonymous': 6, 'nonsynonymous': 2}, 'PB2': {'synonymous': 8, 'nonsynonymous': 0}, 'HA': {'synonymous': 2, 'nonsynonymous': 4}, 'PB1': {'synonymous': 3, 'nonsynonymous': 1}}, 'human': {'HA': {'synonymous': 9, 'nonsynonymous': 11}, 'M2': {'synonymous': 1, 'nonsynonymous': 3}, 'NA': {'synonymous': 6, 'nonsynonymous': 4}, 'NP': {'synonymous': 8, 'nonsynonymous': 5}, 'PA': {'synonymous': 17, 'nonsynonymous': 9}, 'PB2': {'synonymous': 6, 'nonsynonymous': 10}, 'PB1': {'synonymous': 14, 'nonsynonymous': 10}, 'M1': {'synonymous': 5, 'nonsynonymous': 2}, 'NS1': {'synonymous': 3, 'nonsynonymous': 0}, 'NEP': {'synonymous': 0, 'nonsynonymous': 1}}}


In [610]:
# make into a dataframe; this has the total number of SNPs detected per gene per species, not an average
hsnps_df = pd.DataFrame.from_dict(snps_dict["human"], orient="index")
hsnps_df['species'] = "human"
dsnps_df = pd.DataFrame.from_dict(snps_dict["duck"], orient="index")
dsnps_df['species'] = "duck"
snps_df = pd.concat([dsnps_df, hsnps_df])
snps_df.head()

,synonymous,nonsynonymous,species
HA,2,4,duck
M1,0,4,duck
M2,0,1,duck
NA,2,2,duck
NEP,2,1,duck


In [611]:
# take the mean number 
snps_df['mean_synonymous'] = snps_df['synonymous']/6
snps_df['mean_nonsynonymous'] = snps_df['nonsynonymous']/6
snps_df = snps_df.reset_index()
snps_df = snps_df.rename(index=str, columns={'index':'gene'})
snps_df.head()

,gene,synonymous,nonsynonymous,species,mean_synonymous,mean_nonsynonymous
0,HA,2,4,duck,0.333333,0.666667
1,M1,0,4,duck,0.000000,0.666667
2,M2,0,1,duck,0.000000,0.166667
3,NA,2,2,duck,0.333333,0.333333
4,NEP,2,1,duck,0.333333,0.166667


In [612]:
df2 = pd.merge(mean_sites_df, snps_df, on=['gene','species'], how="outer")
df2 = df2.dropna(axis=0, how='any')
df2.head()

,gene,N_sites,S_sites,species,synonymous,nonsynonymous,mean_synonymous,mean_nonsynonymous
0,NEP,286.101039,76.898961,human,0,1,0.000000,0.166667
1,NEP,286.101039,76.898961,human,0,1,0.000000,0.166667
2,NEP,285.992593,77.007407,duck,2,1,0.333333,0.166667
3,NS1,520.724740,154.275260,human,3,0,0.500000,0.000000
4,NS1,521.000000,154.000000,duck,1,3,0.166667,0.500000


In [613]:
# calculate the mean number of nonsynonymous snps per nonsynonymous site and synonymous snps per synonymous site
df2['s_snps/s_site'] = df2['mean_synonymous']/df2['S_sites']
df2['ns_snps/ns_site'] = df2['mean_nonsynonymous']/df2['N_sites']
df2.head()

,gene,N_sites,S_sites,species,synonymous,nonsynonymous,mean_synonymous,mean_nonsynonymous,s_snps/s_site,ns_snps/ns_site
0,NEP,286.101039,76.898961,human,0,1,0.000000,0.166667,0.000000,0.000583
1,NEP,286.101039,76.898961,human,0,1,0.000000,0.166667,0.000000,0.000583
2,NEP,285.992593,77.007407,duck,2,1,0.333333,0.166667,0.004329,0.000583
3,NS1,520.724740,154.275260,human,3,0,0.500000,0.000000,0.003241,0.000000
4,NS1,521.000000,154.000000,duck,1,3,0.166667,0.500000,0.001082,0.000960


In [614]:
# melt to plot 
df2 = pd.melt(df2, id_vars = ["gene","species"], value_vars = ["s_snps/s_site","ns_snps/ns_site"])
df2.head()

,gene,species,variable,value
0,NEP,human,s_snps/s_site,0.000000
1,NEP,human,s_snps/s_site,0.000000
2,NEP,duck,s_snps/s_site,0.004329
3,NS1,human,s_snps/s_site,0.003241
4,NS1,duck,s_snps/s_site,0.001082


In [615]:
%%R -w 800 -h 500 -u px -i df2  # this sets the size of the plot...otherwise, it will go off the page
require(ggplot2)
library(ggplot2)

df2$gene = factor(df2$gene, levels=c("PB2","PB1","PA","HA","NP","NA","M1","M2","NS1","NEP"))

p <- ggplot(data=df2, aes(x=gene, y=value, fill=species)) + 
    geom_col(position="dodge", stat="identity")+
    #geom_errorbar(data=df1, aes(x=gene, ymin=mean-sd, ymax=mean+sd, color=species),position=position_dodge(0.9), stat="identity")+
    facet_wrap(~variable, scales="free")+
    scale_fill_manual(values=c("#99bfaa","#5c3d46"))+
    scale_color_manual(values=c("#99bfaa","#5c3d46"))+
    labs(x="gene",y="SNPs/site")+
    #ggtitle("number of shared sites in random simulations") + 
    scale_y_continuous(limits=c(0,0.008))+
    #scale_x_continuous(limits=c(0,500))+
    theme(plot.title = element_text(size=20, hjust=0.5))+
    theme(panel.grid.major.y=element_line(colour=NA))+
    theme(panel.grid.minor=element_line(colour=NA,size=NA))+    
    theme(strip.background = element_rect(colour=NA, fill=NA))+
    theme(axis.line.x=element_line(colour="black"))+
    theme(axis.line.y=element_line(colour="black"))+theme(strip.text.x=element_text(size=11))+
    theme(axis.title.y=element_text(size=16, vjust=8))+
    theme(axis.title.x=element_text(size=16, vjust=-8))+
    theme(axis.text=element_text(size=16, colour="black"))+
    theme(axis.text.x=element_text(size=16, angle=90))+
    theme(legend.text=element_text(size=16))+theme(legend.title=element_text(size=16, face="plain"))+
    theme(panel.margin=unit(1, "lines"))+theme(plot.margin=unit(c(1,1,1,1),"cm"))+
    theme(legend.key.size=unit(0.7, "cm"))+
    theme(panel.background=element_rect(fill=NA))+
    theme(legend.key=element_rect(fill=NA))

ggsave("snps_per_site-2018-05-11.pdf", p, width = 12, height = 4, device=pdf, path="/Users/lmoncla/Documents/H5N1_Cambodian_outbreak_study/paper-and-figure-drafts/figures-2018-05-08")

/Users/lmoncla/anaconda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning:
  warnings.warn(x, RRuntimeWarning)
/Users/lmoncla/anaconda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning:  Ignoring unknown parameters: stat

  warnings.warn(x, RRuntimeWarning)
